Syntax natürlicher Sprachen, WS 2023/24

# 10 - Übung (Lösung)

In [1]:
import nltk

## Aufgabe 1 - Gewichte und Ableitungswahrscheinlichkeit

#### Betrachten Sie folgende (unvollständige) PCFG.

#### Ergänzen Sie die fehlenden (mit ** markierten) Gewichte und geben Sie die Berechnung für die Ableitungswahrscheinlichkeit durch den Viterbi-Parser an.

In [2]:
grammar = nltk.PCFG.fromstring("""
    S    -> NP VP              [1.0]
    VP   -> TV NP              [0.4]
    VP   -> IV                 [0.3]
    VP   -> DatV NP NP         [0.3]
    TV   -> 'saw'              [1.0]
    IV   -> 'ate'              [1.0]
    DatV -> 'gave'             [1.0]
    NP   -> 'telescopes'       [0.8]
    NP   -> 'Jack'             [0.2]
""")

sent = "Jack saw telescopes"

viterbi_parser = nltk.ViterbiParser(grammar)
for tree in viterbi_parser.parse(sent.split()):
    print(tree)

(S (NP Jack) (VP (TV saw) (NP telescopes))) (p=0.064)


In [3]:
# Berechnung der Ableitungswahrscheinlichkeit:
# 1*0.2*0.4*1*0.8 = 0.064

## Aufgabe 2  - PCFG Parsing
#### Betrachten Sie das folgende PCFG-Parsing:

In [4]:
grammar = nltk.PCFG.fromstring("""
    S -> NP VP [1]
    PP -> P NP [1]
    NP -> DET N [0.5]
    NP -> PRON [0.3]
    NP -> NP PP [0.2]
    VP -> V [0.1]
    VP -> V NP [0.3]
    VP -> VP PP [0.6]

    PRON -> "Er" [1]
    N -> "Mädchen" [0.5] | "Fernglas" [0.5]
    V -> "beobachtet" [1]
    DET -> "das" [0.5] | "dem" [0.5]
    P -> "mit" [1]
""")

sent = "Er beobachtet das Mädchen mit dem Fernglas"

viterbi_parser = nltk.ViterbiParser(grammar)
for tree in viterbi_parser.parse(sent.split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S
  (NP (PRON Er))
  (VP
    (VP (V beobachtet) (NP (DET das) (N Mädchen)))
    (PP (P mit) (NP (DET dem) (N Fernglas))))) (p=0.00084375)
         S                                          
 ┌───────┴─────────────────┐                         
 │                         VP                       
 │               ┌─────────┴─────────┐               
 │               VP                  PP             
 │       ┌───────┴───┐           ┌───┴───┐           
 NP      │           NP          │       NP         
 │       │       ┌───┴─────┐     │   ┌───┴─────┐     
PRON     V      DET        N     P  DET        N    
 │       │       │         │     │   │         │     
 Er  beobachtet das     Mädchen mit dem     Fernglas



### a) Berechnen Sie die Wahrscheinlichkeit für die Ableitung mit dem Viterbi-Parser.

In [5]:
# Berechnung der Ableitungswahrscheinlichkeit:
# 0.00084375 (1*0.3*0.6*0.3*0.5*1*0.5  *  1*1*0.5*0.5*1*0.5*0.5)

### b) Warum findet folgender Parser mehr als eine Ableitung?

In [6]:
parser = nltk.InsideChartParser(grammar)
for tree in parser.parse(sent.split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S
  (NP (PRON Er))
  (VP
    (VP (V beobachtet) (NP (DET das) (N Mädchen)))
    (PP (P mit) (NP (DET dem) (N Fernglas))))) (p=0.00084375)
         S                                          
 ┌───────┴─────────────────┐                         
 │                         VP                       
 │               ┌─────────┴─────────┐               
 │               VP                  PP             
 │       ┌───────┴───┐           ┌───┴───┐           
 NP      │           NP          │       NP         
 │       │       ┌───┴─────┐     │   ┌───┴─────┐     
PRON     V      DET        N     P  DET        N    
 │       │       │         │     │   │         │     
 Er  beobachtet das     Mädchen mit dem     Fernglas

(S
  (NP (PRON Er))
  (VP
    (V beobachtet)
    (NP
      (NP (DET das) (N Mädchen))
      (PP (P mit) (NP (DET dem) (N Fernglas)))))) (p=0.00028125)
         S                                          
 ┌───────┴─────────────────┐                         
 │            

In [7]:
# Antwort:
# InsideChartParser ist ein PCFG-Chart-Parser mit edge queue
# Sortierung nach Wahrscheinlichkeit der Ableitungen

### c) Vergleichen Sie das Tracing folgender probabilistischer Parser und erläutern Sie:

In [8]:
#lowest cost first Chart Parser (Sortierung nach Wahrscheinlichkeit der Zustände/Teilableitungen):
inside_parser = nltk.InsideChartParser(grammar, trace=1)
for tree in inside_parser.parse(sent.split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

  |. . . . . . [-]| [6:7] 'Fernglas'                 [1.0]
  |. . . . . [-] .| [5:6] 'dem'                      [1.0]
  |. . . . [-] . .| [4:5] 'mit'                      [1.0]
  |. . . . [-] . .| [4:5] P  -> 'mit' *              [1.0]
  |. . . . [-> . .| [4:5] PP -> P * NP               [1.0]
  |. . . . > . . .| [4:4] PP -> * P NP               [1.0]
  |. . . . > . . .| [4:4] P  -> * 'mit'              [1.0]
  |. . . [-] . . .| [3:4] 'Mädchen'                  [1.0]
  |. . [-] . . . .| [2:3] 'das'                      [1.0]
  |. [-] . . . . .| [1:2] 'beobachtet'               [1.0]
  |. [-] . . . . .| [1:2] V  -> 'beobachtet' *       [1.0]
  |. > . . . . . .| [1:1] V  -> * 'beobachtet'       [1.0]
  |[-] . . . . . .| [0:1] 'Er'                       [1.0]
  |[-] . . . . . .| [0:1] PRON -> 'Er' *             [1.0]
  |> . . . . . . .| [0:0] PRON -> * 'Er'             [1.0]
  |. . [-] . . . .| [2:3] DET -> 'das' *             [0.5]
  |. . > . . . . .| [2:2] NP -> * DET N              [0.

In [9]:
#best-first Chart Parser (Sortierung nach Länge der Zustände/Teilableitungen):
longest_parser = nltk.LongestChartParser(grammar, trace=1)
for tree in longest_parser.parse(sent.split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

  |. . . . . . [-]| [6:7] 'Fernglas'                 [1.0]
  |. . . . . . [-]| [6:7] N  -> 'Fernglas' *         [0.5]
  |. . . . . [-] .| [5:6] 'dem'                      [1.0]
  |. . . . . [-] .| [5:6] DET -> 'dem' *             [0.5]
  |. . . . . [-> .| [5:6] NP -> DET * N              [0.25]
  |. . . . . [---]| [5:7] NP -> DET N *              [0.125]
  |. . . . . [--->| [5:7] NP -> NP * PP              [0.025]
  |. . . . . [--->| [5:7] S  -> NP * VP              [0.125]
  |. . . . [-] . .| [4:5] 'mit'                      [1.0]
  |. . . . [-] . .| [4:5] P  -> 'mit' *              [1.0]
  |. . . . [-> . .| [4:5] PP -> P * NP               [1.0]
  |. . . . [-----]| [4:7] PP -> P NP *               [0.125]
  |. . . [-] . . .| [3:4] 'Mädchen'                  [1.0]
  |. . . [-] . . .| [3:4] N  -> 'Mädchen' *          [0.5]
  |. . [-] . . . .| [2:3] 'das'                      [1.0]
  |. . [-] . . . .| [2:3] DET -> 'das' *             [0.5]
  |. . [-> . . . .| [2:3] NP -> DET * N        

In [10]:
# Viterbi Parser mit dynamischer Programmierung (effizientes Auffinden der wahrscheinlichsten Ableitung)
viterbi_parser = nltk.ViterbiParser(grammar, trace=2)
for tree in viterbi_parser.parse(sent.split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

Inserting tokens into the most likely constituents table...
   Insert: |=......| Er
   Insert: |.=.....| beobachtet
   Insert: |..=....| das
   Insert: |...=...| Mädchen
   Insert: |....=..| mit
   Insert: |.....=.| dem
   Insert: |......=| Fernglas
Finding the most likely constituents spanning 1 text elements...
   Insert: |=......| PRON -> 'Er' [1.0]
   Insert: |=......| NP -> PRON [0.3]
   Insert: |.=.....| V -> 'beobachtet' [1.0]
   Insert: |.=.....| VP -> V [0.1]
   Insert: |..=....| DET -> 'das' [0.5]
   Insert: |...=...| N -> 'Mädchen' [0.5]
   Insert: |....=..| P -> 'mit' [1.0]
   Insert: |.....=.| DET -> 'dem' [0.5]
   Insert: |......=| N -> 'Fernglas' [0.5]
Finding the most likely constituents spanning 2 text elements...
   Insert: |==.....| S -> NP VP [1.0]
   Insert: |..==...| NP -> DET N [0.5]
   Insert: |.....==| NP -> DET N [0.5]
Finding the most likely constituents spanning 3 text elements...
   Insert: |.===...| VP -> V NP [0.3]
   Insert: |....===| PP -> P NP [1.0]
Fi

In [11]:
#lowest cost first Chart Parser mit Pruning (kein Ergebnis garantiert):
beam_parser = nltk.InsideChartParser(grammar, beam_size=7, trace=1)
for tree in beam_parser.parse(sent.split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

  |. . . . . . [-]| [6:7] 'Fernglas'                 [1.0]
  |. . . . . [-] .| [5:6] 'dem'                      [1.0]
  |. . . . [-] . .| [4:5] 'mit'                      [1.0]
  |. . . . [-] . .| [4:5] P  -> 'mit' *              [1.0]
  |. . . . [-> . .| [4:5] PP -> P * NP               [1.0]
  |. . . . > . . .| [4:4] PP -> * P NP               [1.0]
  |. . . . > . . .| [4:4] P  -> * 'mit'              [1.0]
  |. . . [-] . . .| [3:4] 'Mädchen'                  [1.0]
  |. . [-] . . . .| [2:3] 'das'                      [1.0]
  |. [-] . . . . .| [1:2] 'beobachtet'               [1.0]
  |. [-] . . . . .| [1:2] V  -> 'beobachtet' *       [1.0]
  |. > . . . . . .| [1:1] V  -> * 'beobachtet'       [1.0]
  |[-] . . . . . .| [0:1] 'Er'                       [1.0]
  |[-] . . . . . .| [0:1] PRON -> 'Er' *             [1.0]
  |> . . . . . . .| [0:0] PRON -> * 'Er'             [1.0]
  |. . [-] . . . .| [2:3] DET -> 'das' *             [0.5]
  |. . > . . . . .| [2:2] NP -> * DET N              [0.

## Aufgabe 3 - Abschätzung von Regelwahrscheinlichkeiten

###  a) Herunterladen von Ressourcen und Verarbeitung von CFG-Regeln mit NLTK

#### Laden Sie zunächst die Ressource „corpora/treebank“ mithilfe des NLTK-Download-Managers herunter, falls diese noch nicht installiert ist.

In [12]:
# nltk.download()

 #### Hier ein Beispiel für geparste Sätze in der Penn Treebank:

In [13]:
from nltk.corpus import treebank

for tree in treebank.parsed_sents('wsj_0001.mrg'):
    print(tree)

(S
  (NP-SBJ
    (NP (NNP Pierre) (NNP Vinken))
    (, ,)
    (ADJP (NP (CD 61) (NNS years)) (JJ old))
    (, ,))
  (VP
    (MD will)
    (VP
      (VB join)
      (NP (DT the) (NN board))
      (PP-CLR (IN as) (NP (DT a) (JJ nonexecutive) (NN director)))
      (NP-TMP (NNP Nov.) (CD 29))))
  (. .))
(S
  (NP-SBJ (NNP Mr.) (NNP Vinken))
  (VP
    (VBZ is)
    (NP-PRD
      (NP (NN chairman))
      (PP
        (IN of)
        (NP
          (NP (NNP Elsevier) (NNP N.V.))
          (, ,)
          (NP (DT the) (NNP Dutch) (VBG publishing) (NN group))))))
  (. .))


#### Hier ein Beispiel für die Verarbeitung solcher CFG-Regeln im NLTK:

In [14]:
productions = tree.productions()
productions

[S -> NP-SBJ VP .,
 NP-SBJ -> NNP NNP,
 NNP -> 'Mr.',
 NNP -> 'Vinken',
 VP -> VBZ NP-PRD,
 VBZ -> 'is',
 NP-PRD -> NP PP,
 NP -> NN,
 NN -> 'chairman',
 PP -> IN NP,
 IN -> 'of',
 NP -> NP , NP,
 NP -> NNP NNP,
 NNP -> 'Elsevier',
 NNP -> 'N.V.',
 , -> ',',
 NP -> DT NNP VBG NN,
 DT -> 'the',
 NNP -> 'Dutch',
 VBG -> 'publishing',
 NN -> 'group',
 . -> '.']

In [15]:
productions[0].lhs()

S

In [16]:
productions[0].rhs()

(NP-SBJ, VP, .)

In [17]:
type(productions[0].lhs())

nltk.grammar.Nonterminal

In [18]:
productions[0].lhs() == nltk.grammar.Nonterminal('S')

True

### b) von Daten zu Regelwahrscheinlichkeiten

#### Gegeben sei folgende kontextfreie Grammatik:

In [19]:
grammar = nltk.CFG.fromstring("""
S -> NP VP
VP -> V NP PP
VP -> V NP
NP -> DET N
NP -> NP PP
PP -> P NP

DET -> "the" | "a"
N -> "boy" | "woman" | "telescope"
V -> "saw"
P -> "with"
""")

#### Sie modelliert sehr einfache Sätze der Form `SBJ` *saw* `OBJ` mit optionaler Präpositionalphrase am Ende. Diese Präpositionalphrase kann entweder der näheren Bestimmung des Objekts oder der näheren Bestimmung der in der Verbalphrase ausgedrückten Handlung dienen.


In [20]:
sent = "the boy saw a woman with a telescope"

parser = nltk.ChartParser(grammar,trace=0)
for tree in parser.parse(sent.split()):
    tree.pretty_print(unicodelines=True)

                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌───────┬────┴─────────┐                
     │       │       │              PP              
     │       │       │         ┌────┴───┐            
     NP      │       NP        │        NP          
 ┌───┴───┐   │   ┌───┴────┐    │    ┌───┴──────┐     
DET      N   V  DET       N    P   DET         N    
 │       │   │   │        │    │    │          │     
the     boy saw  a      woman with  a      telescope

                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌────────────┴────┐                     
     │       │                 NP                   
     │       │       ┌─────────┴────┐                
     │       │       │              PP              
     │       │       │         ┌────┴

#### Im folgenden sollen aus einer Treebank Wahrscheinlichkeiten für die einzelnen Regeln extrahiert werden, um diese Ambiguität aufzulösen.

#### Nutzen Sie das im NLTK enthaltene Sample der Penn Treebank (nach Installation unter `nltk.corpus.treebank` zu finden) zunächst zur Identifikation der für eine Disambiguierung nützlichen (Teil-)bäume der Penn Treebank. Dazu zählen Sie zu einem gegebenen Nonterminal als LHS die Produktionen in der Treebank.

#### *Hinweis:* Sie können sich bei der Analyse auf die 30 häufigsten Konstruktionen der Baumbank beschränken.


In [21]:
from collections import defaultdict

def find_relevant_constructions(lhs):
    counter = defaultdict(int)
    #LOESUNG: zähle Produktionen in treebank mit lhs als linker Seite
    for tree in treebank.parsed_sents():  # Format: [(S (NP DET N) (VP V NP))]
        for prod in tree.productions():   # Format: [(S, NP VP), (NP, DET N), (VP, V NP)]
            if prod.lhs() == nltk.grammar.Nonterminal(lhs):
                counter[prod] += 1
    constructions = [ (k, counter[k]) for k in sorted(counter.keys(), key=counter.__getitem__, reverse=True) ]
    return constructions[:30]

find_relevant_constructions("S") #example

[(S -> NP-SBJ VP, 3391),
 (S -> NP-SBJ VP ., 1405),
 (S -> -NONE-, 477),
 (S -> NP-SBJ-1 VP, 371),
 (S -> NP-SBJ-1 VP ., 266),
 (S -> NP-SBJ-2 VP, 201),
 (S -> S , CC S ., 94),
 (S -> CC NP-SBJ VP ., 89),
 (S -> PP-LOC , NP-SBJ VP ., 68),
 (S -> PP , NP-SBJ VP ., 68),
 (S -> NP-SBJ VP . '', 67),
 (S -> NP-SBJ-3 VP, 66),
 (S -> PP-TMP , NP-SBJ VP ., 65),
 (S -> NP-SBJ ADVP VP ., 65),
 (S -> S-TPC-1 , NP-SBJ VP ., 63),
 (S -> NP-SBJ ADVP-TMP VP, 55),
 (S -> `` S-TPC-1 , '' NP-SBJ VP ., 52),
 (S -> NP-SBJ ADVP VP, 51),
 (S -> NP-SBJ NP-PRD, 47),
 (S -> NP-SBJ ADJP-PRD, 46),
 (S -> ADVP , NP-SBJ VP ., 44),
 (S -> SBAR-ADV , NP-SBJ VP ., 43),
 (S -> S : S ., 43),
 (S -> S-TPC-2 , NP-SBJ VP ., 32),
 (S -> S CC S, 28),
 (S -> NP-SBJ ADVP-TMP VP ., 25),
 (S -> ADVP-TMP , NP-SBJ VP ., 24),
 (S -> NP-SBJ-2 VP ., 24),
 (S -> `` NP-SBJ VP . '', 24),
 (S -> S CC S ., 23)]

 #### Für welche Regeln müssen wir die Wahrscheinlichkeiten berechnen, wenn wir mit statistischen Methoden untersuchen wollen, ob PPs häufiger Teil der VP oder Teil der NP sind?

In [22]:
# Antwort:
# VP -> V NP PP
# NP -> NP PP

### c) Abschätzung der Wahrscheinlichkeiten für relevante Regeln

#### Approximieren Sie mittels vergleichbarer Konstruktionen in der Penn Treebank die Wahrscheinlichkeiten der für die Disambiguierung der PP-Ambiguität relevanten Regeln mit Maximum Likelihood Estimation (MLE).

#### (i) Zählen Sie zunächst für die V+NP+PP-Konstruktion, wie oft sie in der Penn Treebank vorkommen und berechnen Sie die relativen Häufigkeiten als Approximation der Regelwahrscheinlichkeiten:

$$P(V, N\!P, P\!P \mid V\!P) = \dfrac{count(V\!P \rightarrow V\:N\!P\:P\!P)}{count(V\!P \rightarrow \setminus*)}$$

**hier (vgl. Grammatik):**

$$= \dfrac{count(V\!P \rightarrow V\:N\!P\:P\!P)}{count(V\!P \rightarrow V\:N\!P\:P\!P) + count(V\!P \rightarrow V\:N\!P)}$$

In [23]:
find_relevant_constructions('VP')

[(VP -> TO VP, 1257),
 (VP -> VB NP, 805),
 (VP -> MD VP, 759),
 (VP -> VBD SBAR, 631),
 (VP -> VBZ VP, 459),
 (VP -> VBD NP, 378),
 (VP -> VBG NP, 375),
 (VP -> VBD VP, 361),
 (VP -> VBP VP, 337),
 (VP -> VBZ NP, 261),
 (VP -> VB VP, 258),
 (VP -> VBN NP, 250),
 (VP -> VP CC VP, 234),
 (VP -> VBD S, 223),
 (VP -> VBZ S, 215),
 (VP -> VBZ SBAR, 197),
 (VP -> VBP NP, 185),
 (VP -> VBN NP PP-CLR, 178),
 (VP -> VBN NP PP, 170),
 (VP -> VBZ NP-PRD, 163),
 (VP -> VB S, 155),
 (VP -> VBN S, 141),
 (VP -> VBP SBAR, 121),
 (VP -> VB PP-CLR, 107),
 (VP -> VBG S, 89),
 (VP -> VBP S, 88),
 (VP -> VB NP PP-CLR, 88),
 (VP -> VBZ ADJP-PRD, 87),
 (VP -> VBN VP, 84),
 (VP -> MD RB VP, 82)]

In [24]:
vp_v_np_pp_frq = 178 + 170 + 88
vp_v_np_without_frq = 805 + 378 + 375 + 261 + 250 + 185 + 163

vp_with_pp = vp_v_np_pp_frq / (vp_v_np_pp_frq + vp_v_np_without_frq)
vp_without = vp_v_np_without_frq / (vp_v_np_pp_frq + vp_v_np_without_frq)

(vp_with_pp, vp_without)

(0.15282159130739573, 0.8471784086926043)

#### (ii) Zählen Sie anschließend, wie oft die NP+PP-Konstruktion in der Penn Treebank vorkommt und berechnen Sie die relativen Häufigkeiten als Approximation der Regelwahrscheinlichkeiten. Das Vorgehen wird in folgender Formel veranschaulicht:

$$P(N\!P, P\!P \mid N\!P) = \dfrac{count(N\!P \rightarrow \:N\!P\:P\!P)}{count(N\!P \rightarrow \setminus*)}$$

**hier:**

$$= \dfrac{count(N\!P \rightarrow \:N\!P\:P\!P)}{count(N\!P \rightarrow \:N\!P\:P\!P) + count(N\!P \rightarrow DET\:N)}$$



In [25]:
find_relevant_constructions('NP')

[(NP -> NP PP, 2188),
 (NP -> DT NN, 2020),
 (NP -> -NONE-, 1225),
 (NP -> NN, 1110),
 (NP -> NNS, 996),
 (NP -> NNP, 837),
 (NP -> DT JJ NN, 740),
 (NP -> NNP NNP, 734),
 (NP -> JJ NNS, 653),
 (NP -> NP SBAR, 409),
 (NP -> JJ NN, 390),
 (NP -> QP -NONE-, 365),
 (NP -> NP PP-LOC, 363),
 (NP -> DT NNS, 358),
 (NP -> CD, 327),
 (NP -> DT NN NN, 313),
 (NP -> NN NNS, 304),
 (NP -> NP CC NP, 289),
 (NP -> NNP NNP NNP, 282),
 (NP -> PRP, 280),
 (NP -> CD NNS, 261),
 (NP -> NP VP, 258),
 (NP -> CD NN, 258),
 (NP -> NNP POS, 230),
 (NP -> NN NN, 219),
 (NP -> $ CD -NONE-, 213),
 (NP -> DT NN POS, 191),
 (NP -> PRP$ NN, 190),
 (NP -> NP NN, 163),
 (NP -> NP , NP, 150)]

In [26]:
np_np_pp_frq = 2188 + 363
np_n_without_frq = 2020 + 358

np_with_pp = np_np_pp_frq / (np_np_pp_frq + np_n_without_frq)
np_without = np_n_without_frq / (np_np_pp_frq + np_n_without_frq)

(np_with_pp, np_without)

(0.5175491986204098, 0.4824508013795902)

### d) Erstellen einer PCFG

#### Die aus den Daten extrahierten relativen Häufigkeiten sollen nun zur Erstellung einer probabilistischen kontextfreien Grammatik (PCFG)  genutzt werden.

In [27]:
(vp_with_pp, vp_without, np_with_pp, np_without)

(0.15282159130739573,
 0.8471784086926043,
 0.5175491986204098,
 0.4824508013795902)

In [28]:
pcfg = f"""
S -> NP VP     [1.0]
VP -> V NP PP  [{vp_with_pp}]
VP -> V NP     [{vp_without}]
NP -> DET N    [{np_without}]
NP -> NP PP    [{np_with_pp}]
PP -> P NP     [1.0]

DET -> "the"     [0.7]
DET -> "a"       [0.3]
N -> "boy"       [0.4]
N -> "woman"     [0.4]
N -> "telescope" [0.2]
V -> "saw"       [1.0]
P -> "with"      [1.0]
"""

grammar = nltk.PCFG.fromstring(pcfg)
print(grammar)

Grammar with 13 productions (start state = S)
    S -> NP VP [1.0]
    VP -> V NP PP [0.152822]
    VP -> V NP [0.847178]
    NP -> DET N [0.482451]
    NP -> NP PP [0.517549]
    PP -> P NP [1.0]
    DET -> 'the' [0.7]
    DET -> 'a' [0.3]
    N -> 'boy' [0.4]
    N -> 'woman' [0.4]
    N -> 'telescope' [0.2]
    V -> 'saw' [1.0]
    P -> 'with' [1.0]


#### Testen Sie Ihre so erstellte Grammatik nun, indem Sie folgenden Satz parsen:

- *the boy saw a woman with a telescope*

In [29]:
sent = "the boy saw a woman with a telescope"

viterbi_parser = nltk.ViterbiParser(grammar)
for tree in viterbi_parser.parse(sent.split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S
  (NP (DET the) (N boy))
  (VP
    (V saw)
    (NP
      (NP (DET a) (N woman))
      (PP (P with) (NP (DET a) (N telescope)))))) (p=9.92604e-05)
                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌────────────┴────┐                     
     │       │                 NP                   
     │       │       ┌─────────┴────┐                
     │       │       │              PP              
     │       │       │         ┌────┴───┐            
     NP      │       NP        │        NP          
 ┌───┴───┐   │   ┌───┴────┐    │    ┌───┴──────┐     
DET      N   V  DET       N    P   DET         N    
 │       │   │   │        │    │    │          │     
the     boy saw  a      woman with  a      telescope



#### Wenn Sie sich die extrahierten Wahrscheinlichkeiten und das disambiguierte Ergebnis ansehen, überrascht Sie dann das Ergebnis der Syntaxanalyse?

In [30]:
# relevante Teilwahrscheinlichkeit VP-Attachment:
vp_with_pp
#P(VP->VP+NP+PP)

0.15282159130739573

In [31]:
# relevante Teilwahrscheinlichkeit NP-Attachment:
vp_without * np_with_pp
#P(VP->VP+NP) * P(NP->NP+PP)

0.4384565065073714

In [32]:
# Antwort:
# Wahrscheinlicheit für NP-Attachment höher

### e) Vergleichen Sie dieses Ergebnis mit der PCFG-Analyse mit folgenden abweichenden Regelwahrscheinlichkeiten. 

#### Warum wird hier, obwohl weiter `vp_with_pp < np_with_pp` gilt, der VP-Attachment-Baum als der wahrscheinlichere ausgewählt? 

In [33]:
vp_with_pp = 0.2
vp_without = 0.8
np_with_pp = 0.22
np_without = 0.78

In [34]:
pcfg = f"""
S -> NP VP     [1.0]
VP -> V NP PP  [{vp_with_pp}]
VP -> V NP     [{vp_without}]
NP -> DET N    [{np_without}]
NP -> NP PP    [{np_with_pp}]
PP -> P NP     [1.0]

DET -> "the"     [0.7]
DET -> "a"       [0.3]
N -> "boy"       [0.4]
N -> "woman"     [0.4]
N -> "telescope" [0.2]
V -> "saw"       [1.0]
P -> "with"      [1.0]
"""

grammar = nltk.PCFG.fromstring(pcfg)
print(grammar)

Grammar with 13 productions (start state = S)
    S -> NP VP [1.0]
    VP -> V NP PP [0.2]
    VP -> V NP [0.8]
    NP -> DET N [0.78]
    NP -> NP PP [0.22]
    PP -> P NP [1.0]
    DET -> 'the' [0.7]
    DET -> 'a' [0.3]
    N -> 'boy' [0.4]
    N -> 'woman' [0.4]
    N -> 'telescope' [0.2]
    V -> 'saw' [1.0]
    P -> 'with' [1.0]


In [35]:
sent = "the boy saw a woman with a telescope"

viterbi_parser = nltk.ViterbiParser(grammar)
for tree in viterbi_parser.parse(sent.split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S
  (NP (DET the) (N boy))
  (VP
    (V saw)
    (NP (DET a) (N woman))
    (PP (P with) (NP (DET a) (N telescope))))) (p=0.000191339)
                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌───────┬────┴─────────┐                
     │       │       │              PP              
     │       │       │         ┌────┴───┐            
     NP      │       NP        │        NP          
 ┌───┴───┐   │   ┌───┴────┐    │    ┌───┴──────┐     
DET      N   V  DET       N    P   DET         N    
 │       │   │   │        │    │    │          │     
the     boy saw  a      woman with  a      telescope



In [36]:
# relevante Teilwahrscheinlichkeit VP-Attachment:
vp_with_pp
#P(VP->VP+NP+PP)

0.2

In [37]:
# relevante Teilwahrscheinlichkeit NP-Attachment:
vp_without * np_with_pp
#P(VP->VP+NP) * P(NP->NP+PP)

0.17600000000000002

In [38]:
# Antwort:
# Anzahl an Regelanwendungen bei NP-Attachment höher (deshalb Gesamtwahrscheinlichkeit geringer)